In [3]:
!pip uninstall Cython -y 
!git clone --depth 1 https://github.com/tensorflow/models

Found existing installation: Cython 3.0.10
Uninstalling Cython-3.0.10:
  Successfully uninstalled Cython-3.0.10


Cloning into 'models'...
Updating files:  75% (2779/3700)
Updating files:  76% (2812/3700)
Updating files:  77% (2849/3700)
Updating files:  78% (2886/3700)
Updating files:  79% (2923/3700)
Updating files:  80% (2960/3700)
Updating files:  81% (2997/3700)
Updating files:  82% (3034/3700)
Updating files:  83% (3071/3700)
Updating files:  84% (3108/3700)
Updating files:  85% (3145/3700)
Updating files:  86% (3182/3700)
Updating files:  87% (3219/3700)
error: unable to create file research/object_detection/samples/configs/ssd_mobilenet_v2_fpnlite_quantized_shared_box_predictor_256x256_depthmultiplier_75_coco14_sync.config: Filename too long
Updating files:  88% (3256/3700)
Updating files:  89% (3293/3700)
Updating files:  90% (3330/3700)
Updating files:  91% (3367/3700)
Updating files:  92% (3404/3700)
Updating files:  93% (3441/3700)
Updating files:  94% (3478/3700)
Updating files:  95% (3515/3700)
Updating files:  96% (3552/3700)
Updating files:  97% (3589/3700)
Updating files:  98% (36

In [4]:
# Copy setup files into models/research folder
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
#cp object_detection/packages/tf2/setup.py .

SyntaxError: invalid syntax (3623106806.py, line 3)

In [6]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
import re
with open('./models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('./models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [7]:
pip install tensorflow==2.8.0


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pyyaml==5.3

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install tensorflow_io==0.23.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
train_record_fname = './train.tfrecord'
val_record_fname = './val.tfrecord'
label_map_pbtxt_fname = './labelmap.pbtxt'

In [3]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    # The centernet model isn't working as of 9/10/22
    #'centernet-mobilenet-v2': {
    #    'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #    'base_pipeline_file': 'pipeline.config',
    #    'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    #}
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [7]:
# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

--2024-05-16 19:41:23--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.221.123, 142.251.221.155, 34.101.5.27, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.221.123|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

     0K .......... .......... .......... .......... ..........  0%  299K 67s
    50K .......... .......... .......... .......... ..........  0%  288K 68s
   100K .......... .......... .......... .......... ..........  0% 3,29M 47s
   150K .......... .......... .......... .......... ..........  0% 3,46M 37s
   200K .......... .......... .......... .......... ..........  1% 3,83M 30s
   250K .......... .......... .......... .......... ..........  1% 6,99M 26s
   300K .......... ....

In [4]:
num_steps = 40000

if chosen_model == 'efficientdet-d0':
  batch_size = 4
else:
  batch_size = 16

In [5]:
# pipeline_fname = './models/mymodel/' + base_pipeline_file
pipeline_fname = base_pipeline_file
fine_tune_checkpoint = './models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)


Total classes: 13


In [6]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

# %cd ./models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)


writing custom configuration file


FileNotFoundError: [Errno 2] No such file or directory: 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config'

In [20]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config'
model_dir = './training/model_output/'

In [22]:
%load_ext tensorboard
%tensorboard --logdir './training/train'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2588), started 21:34:12 ago. (Use '!kill 2588' to kill it.)

In [15]:
import os
import glob

# Check the current working directory
print("Current working directory:", os.getcwd())

# List all files in the directory
print("Files in './training/train/' directory:")
for file in os.listdir('./training/train/'):
    print(file)

# Check for TFRecord files specifically
tfrecord_files = glob.glob('./training/train/train2017-?????-of-00256.tfrecord')
print("TFRecord files found:", tfrecord_files)


Current working directory: C:\Users\Dorino\OneDrive - Universitas Diponegoro\Documents\Kuliah\Semester 6\Bangkit Machine Learning\ML HSE\Model Mandiri
Files in './training/train/' directory:
train.tfrecord
train2017-00001-of-00256.tfrecord
val.tfrecord
TFRecord files found: ['./training/train\\train2017-00001-of-00256.tfrecord']


In [21]:
# Run training!
!python ./models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

^C
